In [1]:
import pandas as pd

# read the data
url = "https://raw.githubusercontent.com/fja05680/sp500/master/S%26P%20500%20Historical%20Components%20%26%20Changes(04-16-2023).csv"
df = pd.read_csv(url)

# split the tickers into separate rows
df = df.assign(tickers=df.tickers.str.split(","))
df = df.explode("tickers")

# create a count column for pivot table
df['count'] = 1

# create the panel data
panel_data = df.pivot_table(index='date', columns='tickers', values='count', aggfunc='sum')
panel_data.fillna(0, inplace=True)

# melt the data to create a "long" format
melted_data = pd.melt(panel_data.reset_index(), id_vars='date', var_name='tickers', value_name='value')

# select rows where value equal 1
melted_data = melted_data[melted_data["value"] == 1]

# drop the "value" column
melted_data.drop('value', axis=1, inplace=True)

# convert the date column to datetime format
melted_data['date'] = pd.to_datetime(melted_data['date'])

# group the tickers by date
grouped_data = melted_data.groupby('date')['tickers'].apply(list)

# select the row of first date of each year
first_dates = grouped_data.groupby(grouped_data.index.year).apply(lambda x: x.iloc[0])

# transpose the resulting data
transposed_data = pd.DataFrame(first_dates.tolist(), index=first_dates.index)
transposed_data = transposed_data.transpose()

# display the resulting data transposed
print(transposed_data)

# save csv file
transposed_data.to_csv("csv_files/sp500_tickers.csv")
     

date   1996   1997   1998   1999   2000   2001   2002   2003  2004  2005  ...  \
0       AAL    AAL  AAMRQ  AAMRQ   AABA      A      A      A     A     A  ...   
1     AAMRQ  AAMRQ   AAPL   AAPL  AAMRQ   AABA   AABA   AABA  AABA  AABA  ...   
2      AAPL   AAPL    ABI    ABI   AAPL  AAMRQ  AAMRQ  AAMRQ  AAPL  AAPL  ...   
3       ABI    ABI    ABS    ABS    ABI   AAPL   AAPL   AAPL   ABC   ABC  ...   
4       ABS    ABS    ABT    ABT    ABS    ABI    ABC    ABC   ABI   ABI  ...   
..      ...    ...    ...    ...    ...    ...    ...    ...   ...   ...  ...   
501    None   None   None   None   None   None   None   None  None  None  ...   
502    None   None   None   None   None   None   None   None  None  None  ...   
503    None   None   None   None   None   None   None   None  None  None  ...   
504    None   None   None   None   None   None   None   None  None  None  ...   
505    None   None   None   None   None   None   None   None  None  None  ...   

date  2014  2015  2016  201

In [ ]:
import pandas as pd
import numpy as np
import time
from alpha_vantage.timeseries import TimeSeries

# read SP500 components 
df_tickers = pd.read_csv("csv_files/sp500_tickers.csv")
df_tickers = df_tickers.rename(columns={"Unnamed: 0": "comp_index"})

# creating a list of all tickers
melted_tickers = pd.melt(df_tickers, id_vars="comp_index", var_name="year", value_name="ticker").dropna()
tickers = melted_tickers["ticker"].unique().tolist()

# api_key 
api_key1 = "XQ40GSMB7OJOL6VQ"
api_key2 = "5X3HSKBVP8LT7590"
api_key3 = "V1PFJOMDHX76TFF8"

# download the adjusted close price for each ticker
data_frames = []
missing_symbols = []

for ticker in tickers:
    try:
        # use api_key1 before exceding number of requests in one day
        ts = TimeSeries(key=api_key1, output_format='pandas')

        print(tickers.index(ticker), ticker)

        data, meta_data = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
        # rename the column and select only the adjusted close price
        data = data[["5. adjusted close"]].rename(columns={"5. adjusted close":"Adj_Close"})
        # create ticker column
        data["ticker"] = ticker
        # get log-returns
        data["log_rtn"] = np.log(data["Adj_Close"]).diff().fillna(0)
        # sort the index in ascending order
        data = data.sort_index()
        # add data to data_frames list 
        data_frames.append(data)
    except ValueError as e:
        error_message = str(e)
        if "Invalid API call" in error_message:
            print(f"Error processing {ticker}: {error_message}")
            # append to removed symbols list
            missing_symbols.append(ticker)
            # do not append the ticker back to the list
            continue
        else:
            print(f"Error processing {ticker}: {error_message}")
            # append the ticker back to the list
            tickers.append(ticker)
            # wait 1 minute before trying again
            time.sleep(60)  
    except ConnectionError as e:
        # use api_key2 before exceding number of requests in one day
        print(f"Error processing {api_key1}: {error_message}")
        # append the ticker back to the list
        tickers.append(ticker)
            
        # new ts
        ts = TimeSeries(key=api_key2, output_format='pandas')

        print(tickers.index(ticker), ticker)

        try:
            data, meta_data = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
            # rename the column and select only the adjusted close price
            data = data[["5. adjusted close"]].rename(columns={"5. adjusted close":"Adj_Close"})
            # create ticker column
            data["ticker"] = ticker
            # get log-returns
            data["log_rtn"] = np.log(data["Adj_Close"]).diff().fillna(0)
            # sort the index in ascending order
            data = data.sort_index()
            # add data to data_frames list 
            data_frames.append(data)
        except ValueError as e:
            error_message = str(e)
            if "Invalid API call" in error_message:
                print(f"Error processing {ticker}: {error_message}")
                # append to removed symbols list
                missing_symbols.append(ticker)
                # do not append the ticker back to the list
                continue
            else:
                print(f"Error processing {ticker}: {error_message}")
                # append the ticker back to the list
                tickers.append(ticker)
                # wait 1 minute before trying again
                time.sleep(60)  
        except ConnectionError as e:
             # use api_key3 before exceding number of requests in one day
            print(f"Error processing {api_key2}: {error_message}")
            # append the ticker back to the list
            tickers.append(ticker)
                
            # new ts
            ts = TimeSeries(key=api_key3, output_format='pandas')

            print(tickers.index(ticker), ticker)

            try:
                data, meta_data = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
                # rename the column and select only the adjusted close price
                data = data[["5. adjusted close"]].rename(columns={"5. adjusted close":"Adj_Close"})
                # create ticker column
                data["ticker"] = ticker
                # get log-returns
                data["log_rtn"] = np.log(data["Adj_Close"]).diff().fillna(0)
                # sort the index in ascending order
                data = data.sort_index()
                # add data to data_frames list 
                data_frames.append(data)
            except ValueError as e:
                error_message = str(e)
                if "Invalid API call" in error_message:
                    print(f"Error processing {ticker}: {error_message}")
                    # append to removed symbols list
                    missing_symbols.append(ticker)
                    # do not append the ticker back to the list
                    continue
                else:
                    print(f"Error processing {ticker}: {error_message}")
                    # append the ticker back to the list
                    tickers.append(ticker)
                    # wait 1 minute before trying again
                    time.sleep(60)  
            except ConnectionError as e:
                # use api_key3 before exceding number of requests in one day
                print(f"Error processing {api_key3}: {error_message}")

# concatenate the data frames into a single data frame
combined_data = pd.concat(data_frames, axis=0)

adj_close_data = combined_data.loc[:, ["ticker", "Adj_Close"]].reset_index().pivot(index="date", columns="ticker", values="Adj_Close")
log_rtn_data = combined_data.loc[:, ["ticker", "log_rtn"]].reset_index().pivot(index="date", columns="ticker", values="log_rtn")

# print tickers not found
missing_symbols_df = pd.DataFrame({"Symbols": missing_symbols})
print(missing_symbols)

# print the first and last 5 rows of the data 
print(adj_close_data.head())
print(log_rtn_data.tail())

# save csv file
missing_symbols_df.to_csv("csv_files/missing_symbols.csv")
adj_close_data.to_csv("csv_files/adj_close.csv")
log_rtn_data.to_csv("csv_files/log_rtn.csv")